# Out-of-State-Contributions: National Analysis

How much out-of-state money have candidates nationally raised in the 2018 election cycle, in absolute and proportional terms, thus far and how does that compare with this point in the 2014 and 2010 cycles?

In [1]:
import numpy as np
import pandas as pd

%load_ext jupyternotify

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)
pd.options.display.float_format = "{:,.2f}".format # Format floats

<IPython.core.display.Javascript object>

Import contributions data.

In [2]:
%%notify
contributions = pd.read_csv("data/contributions.csv")
contributions.info()

/home/jyerardi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (14,17,18,19,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6514274 entries, 0 to 6514273
Data columns (total 23 columns):
candidate                 object
candidate_id              float64
year                      float64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      object
contributor_street        object
contributor_city          object
contributor_state         object
contributor_zip           float64
in_out_state              object
no_veto                   object
office                    object
latest_month              object
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: float64(4), object(19)
memory usage: 1.1+ GB


<IPython.core.display.Javascript object>

Convert the candidate ID and year columns to integer data type, the contribution date and latest month columns to datetime data type and the contributor zip column to object data type.

In [3]:
contributions["candidate_id"] = pd.to_numeric(contributions["candidate_id"], errors="coerce", downcast="integer")
contributions["year"] = pd.to_numeric(contributions["year"], errors="coerce", downcast="integer")
contributions["date"] = pd.to_datetime(contributions["date"], errors="coerce")
contributions["latest_month"] = pd.to_datetime(contributions["latest_month"], errors="coerce")
contributions["contributor_zip"] = contributions["contributor_zip"].astype(object)
contributions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6514274 entries, 0 to 6514273
Data columns (total 23 columns):
candidate                 object
candidate_id              int32
year                      int16
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      datetime64[ns]
contributor_street        object
contributor_city          object
contributor_state         object
contributor_zip           object
in_out_state              object
no_veto                   object
office                    object
latest_month              datetime64[ns]
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: datetime64[ns](2), float64(1), int16(1), int32(1), object(18)
memory usage: 1.1+ GB


Import full cycle contributions data.

In [4]:
%%notify
contributions_full_cycles = pd.read_csv("data/contributions_full_cycles.csv")
contributions_full_cycles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9029907 entries, 0 to 9029906
Data columns (total 17 columns):
candidate_id           float64
candidate              object
election_status        object
party                  object
state                  object
year                   float64
office                 object
contributor            object
amount                 float64
date                   object
contributor_street     object
contributor_city       object
contributor_state      object
contributor_zip        float64
in_out_state           object
standardized_office    object
standardized_status    object
dtypes: float64(4), object(13)
memory usage: 1.1+ GB


<IPython.core.display.Javascript object>

Convert the candidate ID and year columns to integer data type, the contribution date column to datetime data type and the contributor zip column to object data type.

In [5]:
contributions_full_cycles["candidate_id"] = pd.to_numeric(contributions_full_cycles["candidate_id"], errors="coerce", downcast="integer")
contributions_full_cycles["year"] = pd.to_numeric(contributions_full_cycles["year"], errors="coerce", downcast="integer")
contributions_full_cycles["date"] = pd.to_datetime(contributions_full_cycles["date"], errors="coerce")
contributions_full_cycles["contributor_zip"] = contributions_full_cycles["contributor_zip"].astype(object)
contributions_full_cycles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9029907 entries, 0 to 9029906
Data columns (total 17 columns):
candidate_id           int32
candidate              object
election_status        object
party                  object
state                  object
year                   int16
office                 object
contributor            object
amount                 float64
date                   datetime64[ns]
contributor_street     object
contributor_city       object
contributor_state      object
contributor_zip        object
in_out_state           object
standardized_office    object
standardized_status    object
dtypes: datetime64[ns](1), float64(1), int16(1), int32(1), object(13)
memory usage: 1.1+ GB


## Calculate out-of-state contributions by party and year

Group by year and party and in-vs.-out-of-state contribution status and sum contributions.

In [6]:
contributions_by_party = contributions.groupby(["year", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_party.head()

,year,party,in_out_state,amount
0,2010,Democratic,in-state,"479,980,470.34"
1,2010,Democratic,out-of-state,"60,701,273.05"
2,2010,Democratic,unknown,"1,575,368.26"
3,2010,Nonpartisan,in-state,"941,726.00"
4,2010,Nonpartisan,out-of-state,"136,753.70"


Pivot dataframe to aggregate each party's data in a single row.

In [7]:
contributions_by_party = pd.pivot_table(contributions_by_party, index=["party"], columns=["year", "in_out_state"]).reset_index()
contributions_by_party

party         amount                             \
year                                2010                              
in_out_state                    in-state  out-of-state      unknown   
0              Democratic 479,980,470.34 60,701,273.05 1,575,368.26   
1             Nonpartisan     941,726.00    136,753.70   -19,688.72   
2              Republican 663,193,378.15 51,035,306.97   763,764.14   
3             Third-Party   6,276,829.73    662,664.02 1,536,886.63   
4                 Unknown            nan           nan          nan   

                                                                        \
year                   2014                                       2018   
in_out_state       in-state  out-of-state       unknown       in-state   
0            415,887,179.38 71,931,756.87  6,061,131.29 682,259,324.37   
1              1,617,096.45    156,743.05      4,068.00   1,376,337.33   
2            467,677,904.26 75,848,880.89 11,001,146.49 770,150,186.19   
3             12,252,912.67    619,901.81     37,163.76   3,961,221.84   
4                       nan           nan           nan      28,042.01   

                                          
year                                      
in_out_state  out-of-state       unknown  
0            98,540,154.13 10,130,470.91  
1               310,506.30     -3,156.23  
2            70,514,499.59  9,679,414.25  
3               409,923.18    520,433.33  
4                 7,944.09           nan

Some records have no contributions for certain categories. Let's set those values equal to zero to be sure any calculations we run on them are correct.

In [8]:
contributions_by_party.fillna(0, inplace=True)

Flatten the resulting dataframe's multi-index columns.

In [9]:
contributions_by_party.columns = ["party", "10_in_state", "10_out_of_state", "10_unknown",
                                  "14_in_state", "14_out_of_state", "14_unknown",
                                  "18_in_state", "18_out_of_state", "18_unknown"
                                  ]
contributions_by_party

,party,10_in_state,10_out_of_state,10_unknown,14_in_state,14_out_of_state,14_unknown,18_in_state,18_out_of_state,18_unknown
0,Democratic,"479,980,470.34","60,701,273.05","1,575,368.26","415,887,179.38","71,931,756.87","6,061,131.29","682,259,324.37","98,540,154.13","10,130,470.91"
1,Nonpartisan,"941,726.00","136,753.70","-19,688.72","1,617,096.45","156,743.05","4,068.00","1,376,337.33","310,506.30","-3,156.23"
2,Republican,"663,193,378.15","51,035,306.97","763,764.14","467,677,904.26","75,848,880.89","11,001,146.49","770,150,186.19","70,514,499.59","9,679,414.25"
3,Third-Party,"6,276,829.73","662,664.02","1,536,886.63","12,252,912.67","619,901.81","37,163.76","3,961,221.84","409,923.18","520,433.33"
4,Unknown,0.00,0.00,0.00,0.00,0.00,0.00,"28,042.01","7,944.09",0.00


Calculate the total contributions by cycle.

In [10]:
contributions_by_party["18_total"] = contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"]
contributions_by_party["14_total"] = contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"]
contributions_by_party["10_total"] = contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"]
contributions_by_party = contributions_by_party[["party", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]

Calculate the proportion of in-state, out-of-state and unknown contributions by cycle.

In [11]:
contributions_by_party["pct_18_in_state"] = contributions_by_party["18_in_state"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_18_out_of_state"] = contributions_by_party["18_out_of_state"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_18_unknown"] = contributions_by_party["18_unknown"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_14_in_state"] = contributions_by_party["14_in_state"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_14_out_of_state"] = contributions_by_party["14_out_of_state"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_14_unknown"] = contributions_by_party["14_unknown"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_10_in_state"] = contributions_by_party["10_in_state"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party["pct_10_out_of_state"] = contributions_by_party["10_out_of_state"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party["pct_10_unknown"] = contributions_by_party["10_unknown"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party

/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,party,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,Democratic,"682,259,324.37","98,540,154.13","10,130,470.91","790,929,949.41","415,887,179.38","71,931,756.87","6,061,131.29","493,880,067.54","479,980,470.34","60,701,273.05","1,575,368.26","542,257,111.65",0.86,0.12,0.01,0.84,0.15,0.01,0.89,0.11,0.00
1,Nonpartisan,"1,376,337.33","310,506.30","-3,156.23","1,683,687.40","1,617,096.45","156,743.05","4,068.00","1,777,907.50","941,726.00","136,753.70","-19,688.72","1,058,790.98",0.82,0.18,-0.00,0.91,0.09,0.00,0.89,0.13,-0.02
2,Republican,"770,150,186.19","70,514,499.59","9,679,414.25","850,344,100.03","467,677,904.26","75,848,880.89","11,001,146.49","554,527,931.64","663,193,378.15","51,035,306.97","763,764.14","714,992,449.26",0.91,0.08,0.01,0.84,0.14,0.02,0.93,0.07,0.00
3,Third-Party,"3,961,221.84","409,923.18","520,433.33","4,891,578.35","12,252,912.67","619,901.81","37,163.76","12,909,978.24","6,276,829.73","662,664.02","1,536,886.63","8,476,380.38",0.81,0.08,0.11,0.95,0.05,0.00,0.74,0.08,0.18
4,Unknown,"28,042.01","7,944.09",0.00,"35,986.10",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.78,0.22,0.00,nan,nan,nan,nan,nan,nan


## Calculate out-of-state contributions by party and year for the complete cycle

Group by year and party and in-vs.-out-of-state contribution status and sum contributions.

In [12]:
contributions_by_party_full_cycles = contributions_full_cycles.groupby(["year", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_party_full_cycles.head()

,year,party,in_out_state,amount
0,2010,Democratic,in-state,"782,603,976.89"
1,2010,Democratic,out-of-state,"98,979,342.93"
2,2010,Democratic,unknown,"5,874,560.04"
3,2010,Nonpartisan,in-state,"1,393,839.63"
4,2010,Nonpartisan,out-of-state,"197,886.63"


Pivot dataframe to aggregate each party's data in a single row.

In [13]:
contributions_by_party_full_cycles = pd.pivot_table(contributions_by_party_full_cycles, index=["party"], columns=["year", "in_out_state"]).reset_index()
contributions_by_party_full_cycles

party           amount                             \
year                                  2010                              
in_out_state                      in-state  out-of-state      unknown   
0              Democratic   782,603,976.89 98,979,342.93 5,874,560.04   
1             Nonpartisan     1,393,839.63    197,886.63   -16,406.72   
2              Republican 1,050,695,215.16 98,930,400.21 3,278,640.99   
3             Third-Party    16,748,739.09  1,553,752.31 5,213,802.52   
4                 Unknown              nan           nan          nan   

                                                                         \
year                   2014                                        2018   
in_out_state       in-state   out-of-state       unknown       in-state   
0            687,915,466.00 115,967,023.41  9,128,513.19 694,587,854.89   
1              3,559,696.98     464,716.75     10,268.00   1,376,337.33   
2            864,954,477.29 124,538,245.24 14,348,298.86 789,119,093.98   
3             19,055,899.25   1,099,466.87     83,266.73   4,000,287.22   
4                       nan            nan           nan      28,042.01   

                                           
year                                       
in_out_state   out-of-state       unknown  
0            103,472,177.49 10,611,612.17  
1                310,506.30     -3,156.23  
2             74,486,861.71 11,468,468.46  
3                413,133.18    521,000.47  
4                  7,944.09           nan

Some records have no contributions for certain categories. Let's set those values equal to zero to be sure any calculations we run on them are correct.

In [14]:
contributions_by_party_full_cycles.fillna(0, inplace=True)

Flatten the resulting dataframe's multi-index columns.

In [15]:
contributions_by_party_full_cycles.columns = ["party", "10_in_state", "10_out_of_state", "10_unknown",
                                  "14_in_state", "14_out_of_state", "14_unknown",
                                  "18_in_state", "18_out_of_state", "18_unknown"
                                  ]
contributions_by_party_full_cycles

,party,10_in_state,10_out_of_state,10_unknown,14_in_state,14_out_of_state,14_unknown,18_in_state,18_out_of_state,18_unknown
0,Democratic,"782,603,976.89","98,979,342.93","5,874,560.04","687,915,466.00","115,967,023.41","9,128,513.19","694,587,854.89","103,472,177.49","10,611,612.17"
1,Nonpartisan,"1,393,839.63","197,886.63","-16,406.72","3,559,696.98","464,716.75","10,268.00","1,376,337.33","310,506.30","-3,156.23"
2,Republican,"1,050,695,215.16","98,930,400.21","3,278,640.99","864,954,477.29","124,538,245.24","14,348,298.86","789,119,093.98","74,486,861.71","11,468,468.46"
3,Third-Party,"16,748,739.09","1,553,752.31","5,213,802.52","19,055,899.25","1,099,466.87","83,266.73","4,000,287.22","413,133.18","521,000.47"
4,Unknown,0.00,0.00,0.00,0.00,0.00,0.00,"28,042.01","7,944.09",0.00


Calculate the total contributions by cycle.

In [16]:
contributions_by_party_full_cycles["18_total"] = contributions_by_party_full_cycles["18_in_state"] + contributions_by_party_full_cycles["18_out_of_state"] + contributions_by_party_full_cycles["18_unknown"]
contributions_by_party_full_cycles["14_total"] = contributions_by_party_full_cycles["14_in_state"] + contributions_by_party_full_cycles["14_out_of_state"] + contributions_by_party_full_cycles["14_unknown"]
contributions_by_party_full_cycles["10_total"] = contributions_by_party_full_cycles["10_in_state"] + contributions_by_party_full_cycles["10_out_of_state"] + contributions_by_party_full_cycles["10_unknown"]
contributions_by_party_full_cycles = contributions_by_party_full_cycles[["party", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]

Calculate the proportion of in-state, out-of-state and unknown contributions by cycle.

In [17]:
contributions_by_party_full_cycles["pct_18_in_state"] = contributions_by_party_full_cycles["18_in_state"] / (contributions_by_party_full_cycles["18_in_state"] + contributions_by_party_full_cycles["18_out_of_state"] + contributions_by_party_full_cycles["18_unknown"])
contributions_by_party_full_cycles["pct_18_out_of_state"] = contributions_by_party_full_cycles["18_out_of_state"] / (contributions_by_party_full_cycles["18_in_state"] + contributions_by_party_full_cycles["18_out_of_state"] + contributions_by_party_full_cycles["18_unknown"])
contributions_by_party_full_cycles["pct_18_unknown"] = contributions_by_party_full_cycles["18_unknown"] / (contributions_by_party_full_cycles["18_in_state"] + contributions_by_party_full_cycles["18_out_of_state"] + contributions_by_party_full_cycles["18_unknown"])
contributions_by_party_full_cycles["pct_14_in_state"] = contributions_by_party_full_cycles["14_in_state"] / (contributions_by_party_full_cycles["14_in_state"] + contributions_by_party_full_cycles["14_out_of_state"] + contributions_by_party_full_cycles["14_unknown"])
contributions_by_party_full_cycles["pct_14_out_of_state"] = contributions_by_party_full_cycles["14_out_of_state"] / (contributions_by_party_full_cycles["14_in_state"] + contributions_by_party_full_cycles["14_out_of_state"] + contributions_by_party_full_cycles["14_unknown"])
contributions_by_party_full_cycles["pct_14_unknown"] = contributions_by_party_full_cycles["14_unknown"] / (contributions_by_party_full_cycles["14_in_state"] + contributions_by_party_full_cycles["14_out_of_state"] + contributions_by_party_full_cycles["14_unknown"])
contributions_by_party_full_cycles["pct_10_in_state"] = contributions_by_party_full_cycles["10_in_state"] / (contributions_by_party_full_cycles["10_in_state"] + contributions_by_party_full_cycles["10_out_of_state"] + contributions_by_party_full_cycles["10_unknown"])
contributions_by_party_full_cycles["pct_10_out_of_state"] = contributions_by_party_full_cycles["10_out_of_state"] / (contributions_by_party_full_cycles["10_in_state"] + contributions_by_party_full_cycles["10_out_of_state"] + contributions_by_party_full_cycles["10_unknown"])
contributions_by_party_full_cycles["pct_10_unknown"] = contributions_by_party_full_cycles["10_unknown"] / (contributions_by_party_full_cycles["10_in_state"] + contributions_by_party_full_cycles["10_out_of_state"] + contributions_by_party_full_cycles["10_unknown"])
contributions_by_party_full_cycles

/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,party,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,Democratic,"694,587,854.89","103,472,177.49","10,611,612.17","808,671,644.55","687,915,466.00","115,967,023.41","9,128,513.19","813,011,002.60","782,603,976.89","98,979,342.93","5,874,560.04","887,457,879.86",0.86,0.13,0.01,0.85,0.14,0.01,0.88,0.11,0.01
1,Nonpartisan,"1,376,337.33","310,506.30","-3,156.23","1,683,687.40","3,559,696.98","464,716.75","10,268.00","4,034,681.73","1,393,839.63","197,886.63","-16,406.72","1,575,319.54",0.82,0.18,-0.00,0.88,0.12,0.00,0.88,0.13,-0.01
2,Republican,"789,119,093.98","74,486,861.71","11,468,468.46","875,074,424.15","864,954,477.29","124,538,245.24","14,348,298.86","1,003,841,021.39","1,050,695,215.16","98,930,400.21","3,278,640.99","1,152,904,256.36",0.90,0.09,0.01,0.86,0.12,0.01,0.91,0.09,0.00
3,Third-Party,"4,000,287.22","413,133.18","521,000.47","4,934,420.87","19,055,899.25","1,099,466.87","83,266.73","20,238,632.85","16,748,739.09","1,553,752.31","5,213,802.52","23,516,293.92",0.81,0.08,0.11,0.94,0.05,0.00,0.71,0.07,0.22
4,Unknown,"28,042.01","7,944.09",0.00,"35,986.10",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.78,0.22,0.00,nan,nan,nan,nan,nan,nan


## Calculate out-of-state contributions by office and year

Group by year, standardized office and in-vs.-out-of-state contribution status and sum contributions.

In [18]:
contributions_by_office = contributions.groupby(["year", "standardized_office", "in_out_state"])["amount"].sum().reset_index()
contributions_by_office.head()

,year,standardized_office,in_out_state,amount
0,2010,GOVERNOR/LIEUTENANT GOVERNOR,in-state,"694,989,888.64"
1,2010,GOVERNOR/LIEUTENANT GOVERNOR,out-of-state,"67,431,628.46"
2,2010,GOVERNOR/LIEUTENANT GOVERNOR,unknown,"2,106,241.41"
3,2010,STATE HOUSE/ASSEMBLY/SENATE,in-state,"455,402,515.58"
4,2010,STATE HOUSE/ASSEMBLY/SENATE,out-of-state,"45,104,369.28"


Pivot the dataframe to aggregate each office's data in a single row.

In [19]:
contributions_by_office = pd.pivot_table(contributions_by_office, index=["standardized_office"], columns=["year", "in_out_state"]).reset_index()
contributions_by_office

standardized_office         amount                \
year                                                 2010                 
in_out_state                                     in-state  out-of-state   
0             GOVERNOR/LIEUTENANT GOVERNOR 694,989,888.64 67,431,628.46   
1              STATE HOUSE/ASSEMBLY/SENATE 455,402,515.58 45,104,369.28   

                                                                      \
year                                2014                               
in_out_state      unknown       in-state  out-of-state       unknown   
0            2,106,241.41 411,270,071.99 92,764,840.66 10,249,495.27   
1            1,750,088.90 486,165,020.77 55,792,441.96  6,854,014.27   

                                                         
year                   2018                              
in_out_state       in-state  out-of-state       unknown  
0            926,392,772.24 99,735,006.44  7,668,123.95  
1            531,382,339.50 70,048,020.85 12,659,038.31

Flatten the resulting dataframe's multi-index columns.

In [20]:
contributions_by_office.columns = ["standardized_office", "10_in_state", "10_out_of_state", "10_unknown",
                                  "14_in_state", "14_out_of_state", "14_unknown",
                                  "18_in_state", "18_out_of_state", "18_unknown"
                                  ]
contributions_by_office

,standardized_office,10_in_state,10_out_of_state,10_unknown,14_in_state,14_out_of_state,14_unknown,18_in_state,18_out_of_state,18_unknown
0,GOVERNOR/LIEUTENANT GOVERNOR,"694,989,888.64","67,431,628.46","2,106,241.41","411,270,071.99","92,764,840.66","10,249,495.27","926,392,772.24","99,735,006.44","7,668,123.95"
1,STATE HOUSE/ASSEMBLY/SENATE,"455,402,515.58","45,104,369.28","1,750,088.90","486,165,020.77","55,792,441.96","6,854,014.27","531,382,339.50","70,048,020.85","12,659,038.31"


Calculate the total contributions by cycle.

In [21]:
contributions_by_office["18_total"] = contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"]
contributions_by_office["14_total"] = contributions_by_office["14_in_state"] + contributions_by_office["14_out_of_state"] + contributions_by_office["14_unknown"]
contributions_by_office["10_total"] = contributions_by_office["10_in_state"] + contributions_by_office["10_out_of_state"] + contributions_by_office["10_unknown"]
contributions_by_office = contributions_by_office[["standardized_office", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]
contributions_by_office

,standardized_office,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total
0,GOVERNOR/LIEUTENANT GOVERNOR,"926,392,772.24","99,735,006.44","7,668,123.95","1,033,795,902.63","411,270,071.99","92,764,840.66","10,249,495.27","514,284,407.92","694,989,888.64","67,431,628.46","2,106,241.41","764,527,758.51"
1,STATE HOUSE/ASSEMBLY/SENATE,"531,382,339.50","70,048,020.85","12,659,038.31","614,089,398.66","486,165,020.77","55,792,441.96","6,854,014.27","548,811,477.00","455,402,515.58","45,104,369.28","1,750,088.90","502,256,973.76"


Calculate the proportion of in-state, out-of-state and unknown contributions.

In [22]:
contributions_by_office["pct_18_in_state"] = contributions_by_office["18_in_state"] / (contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"])
contributions_by_office["pct_18_out_of_state"] = contributions_by_office["18_out_of_state"] / (contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"])
contributions_by_office["pct_18_unknown"] = contributions_by_office["18_unknown"] / (contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"])
contributions_by_office["pct_14_in_state"] = contributions_by_office["14_in_state"] / (contributions_by_office["14_in_state"] + contributions_by_office["14_out_of_state"] + contributions_by_office["14_unknown"])
contributions_by_office["pct_14_out_of_state"] = contributions_by_office["14_out_of_state"] / (contributions_by_office["14_in_state"] + contributions_by_office["14_out_of_state"] + contributions_by_office["14_unknown"])
contributions_by_office["pct_14_unknown"] = contributions_by_office["14_unknown"] / (contributions_by_office["14_in_state"] + contributions_by_office["14_out_of_state"] + contributions_by_office["14_unknown"])
contributions_by_office["pct_10_in_state"] = contributions_by_office["10_in_state"] / (contributions_by_office["10_in_state"] + contributions_by_office["10_out_of_state"] + contributions_by_office["10_unknown"])
contributions_by_office["pct_10_out_of_state"] = contributions_by_office["10_out_of_state"] / (contributions_by_office["10_in_state"] + contributions_by_office["10_out_of_state"] + contributions_by_office["10_unknown"])
contributions_by_office["pct_10_unknown"] = contributions_by_office["10_unknown"] / (contributions_by_office["10_in_state"] + contributions_by_office["10_out_of_state"] + contributions_by_office["10_unknown"])
contributions_by_office

/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,standardized_office,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,GOVERNOR/LIEUTENANT GOVERNOR,"926,392,772.24","99,735,006.44","7,668,123.95","1,033,795,902.63","411,270,071.99","92,764,840.66","10,249,495.27","514,284,407.92","694,989,888.64","67,431,628.46","2,106,241.41","764,527,758.51",0.90,0.10,0.01,0.80,0.18,0.02,0.91,0.09,0.00
1,STATE HOUSE/ASSEMBLY/SENATE,"531,382,339.50","70,048,020.85","12,659,038.31","614,089,398.66","486,165,020.77","55,792,441.96","6,854,014.27","548,811,477.00","455,402,515.58","45,104,369.28","1,750,088.90","502,256,973.76",0.87,0.11,0.02,0.89,0.10,0.01,0.91,0.09,0.00


## Calculate 2018 out-of-state contributions by redistricting role

Filter the contributions data to the 2018 cycle.

In [23]:
contributions_18 = contributions[contributions["year"] == 2018]
contributions_18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2512343 entries, 0 to 2512342
Data columns (total 23 columns):
candidate                 object
candidate_id              int32
year                      int16
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      datetime64[ns]
contributor_street        object
contributor_city          object
contributor_state         object
contributor_zip           object
in_out_state              object
no_veto                   object
office                    object
latest_month              datetime64[ns]
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: datetime64[ns](2), float64(1), int16(1), int32(1), object(18)
memory usage: 436.1+ MB


Group by redistricting role and in-vs.-out-of-state contribution status and sum contributions.

In [24]:
contributions_by_redistricting = contributions.groupby(["redistricting_role", "in_out_state"])["amount"].sum().reset_index()
contributions_by_redistricting

,redistricting_role,in_out_state,amount
0,N,in-state,"236,727,923.38"
1,N,out-of-state,"28,458,559.52"
2,N,unknown,"4,294,409.53"
3,Y,in-state,"1,221,047,188.36"
4,Y,out-of-state,"141,324,467.77"
5,Y,unknown,"16,032,752.73"


Pivot dataframe to aggregate each role's data in a single row.

In [25]:
contributions_by_redistricting = pd.pivot_table(contributions_by_redistricting, index=["redistricting_role"], columns=["in_out_state"]).reset_index()
contributions_by_redistricting

redistricting_role           amount                             
in_out_state                            in-state   out-of-state       unknown
0                             N   236,727,923.38  28,458,559.52  4,294,409.53
1                             Y 1,221,047,188.36 141,324,467.77 16,032,752.73

Flatten the resulting dataframe's multi-index columns.

In [26]:
contributions_by_redistricting.columns = ["redistricting_role", "in_state", "out_of_state", "unknown"]
contributions_by_redistricting

,redistricting_role,in_state,out_of_state,unknown
0,N,"236,727,923.38","28,458,559.52","4,294,409.53"
1,Y,"1,221,047,188.36","141,324,467.77","16,032,752.73"


Calculate the total contributions by redistricting role.

In [27]:
contributions_by_redistricting["total"] = contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"]

Calculate the proportion of in-state, out-of-state and unknown contributions by redistricting role.

In [28]:
contributions_by_redistricting["pct_in_state"] = contributions_by_redistricting["in_state"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting["pct_out_of_state"] = contributions_by_redistricting["out_of_state"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting["pct_unknown"] = contributions_by_redistricting["unknown"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting

,redistricting_role,in_state,out_of_state,unknown,total,pct_in_state,pct_out_of_state,pct_unknown
0,N,"236,727,923.38","28,458,559.52","4,294,409.53","269,480,892.43",0.88,0.11,0.02
1,Y,"1,221,047,188.36","141,324,467.77","16,032,752.73","1,378,404,408.86",0.89,0.10,0.01


## Calculate contributions by candidate status and year

Group by year and candidate status and in-vs.-out-of-state contribution status and sum contributions.

In [29]:
contributions_by_status = contributions.groupby(["year", "standardized_status", "in_out_state"])["amount"].sum().reset_index()
contributions_by_status

,year,standardized_status,in_out_state,amount
0,2010,ADVANCED TO GENERAL,in-state,"904,422,180.55"
1,2010,ADVANCED TO GENERAL,out-of-state,"95,032,126.24"
2,2010,ADVANCED TO GENERAL,unknown,"3,128,068.10"
3,2010,DID NOT ADVANCE,in-state,"245,970,223.67"
4,2010,DID NOT ADVANCE,out-of-state,"17,503,871.50"
5,2010,DID NOT ADVANCE,unknown,"728,262.21"
6,2014,ADVANCED TO GENERAL,in-state,"766,347,377.80"
7,2014,ADVANCED TO GENERAL,out-of-state,"129,404,221.01"
8,2014,ADVANCED TO GENERAL,unknown,"11,824,133.19"
9,2014,DID NOT ADVANCE,in-state,"131,087,714.96"


Pivot dataframe to aggregate each candidate status' data in a single row.

In [30]:
contributions_by_status = pd.pivot_table(contributions_by_status, index=["standardized_status"], columns=["year", "in_out_state"]).reset_index()
contributions_by_status

standardized_status         amount                             \
year                                        2010                              
in_out_state                            in-state  out-of-state      unknown   
0             ADVANCED TO GENERAL 904,422,180.55 95,032,126.24 3,128,068.10   
1                 DID NOT ADVANCE 245,970,223.67 17,503,871.50   728,262.21   

                                                                           \
year                   2014                                          2018   
in_out_state       in-state   out-of-state       unknown         in-state   
0            766,347,377.80 129,404,221.01 11,824,133.19 1,033,494,016.03   
1            131,087,714.96  19,153,061.61  5,279,376.35   424,281,095.71   

                                           
year                                       
in_out_state   out-of-state       unknown  
0            132,135,037.53 15,165,680.27  
1             37,647,989.76  5,161,481.99

Flatten the resulting dataframe's multi-index columns.

In [31]:
contributions_by_status.columns = ["standardized_status", "10_in_state", "10_out_of_state", "10_unknown",
                                   "14_in_state", "14_out_of_state", "14_unknown",
                                   "18_in_state", "18_out_of_state", "18_unknown"
                                  ]
contributions_by_status

,standardized_status,10_in_state,10_out_of_state,10_unknown,14_in_state,14_out_of_state,14_unknown,18_in_state,18_out_of_state,18_unknown
0,ADVANCED TO GENERAL,"904,422,180.55","95,032,126.24","3,128,068.10","766,347,377.80","129,404,221.01","11,824,133.19","1,033,494,016.03","132,135,037.53","15,165,680.27"
1,DID NOT ADVANCE,"245,970,223.67","17,503,871.50","728,262.21","131,087,714.96","19,153,061.61","5,279,376.35","424,281,095.71","37,647,989.76","5,161,481.99"


Calculate the total contributions by cycle.

In [32]:
contributions_by_status["18_total"] = contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"]
contributions_by_status["14_total"] = contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"]
contributions_by_status["10_total"] = contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"]
contributions_by_status = contributions_by_status[["standardized_status", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]

Calculate the proportion of in-state, out-of-state and unknown contributions by cycle.

In [33]:
contributions_by_status["pct_18_in_state"] = contributions_by_status["18_in_state"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_18_out_of_state"] = contributions_by_status["18_out_of_state"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_18_unknown"] = contributions_by_status["18_unknown"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_14_in_state"] = contributions_by_status["14_in_state"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_14_out_of_state"] = contributions_by_status["14_out_of_state"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_14_unknown"] = contributions_by_status["14_unknown"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_10_in_state"] = contributions_by_status["10_in_state"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status["pct_10_out_of_state"] = contributions_by_status["10_out_of_state"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status["pct_10_unknown"] = contributions_by_status["10_unknown"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status

/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jyerardi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

,standardized_status,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,ADVANCED TO GENERAL,"1,033,494,016.03","132,135,037.53","15,165,680.27","1,180,794,733.83","766,347,377.80","129,404,221.01","11,824,133.19","907,575,732.00","904,422,180.55","95,032,126.24","3,128,068.10","1,002,582,374.89",0.88,0.11,0.01,0.84,0.14,0.01,0.90,0.09,0.00
1,DID NOT ADVANCE,"424,281,095.71","37,647,989.76","5,161,481.99","467,090,567.46","131,087,714.96","19,153,061.61","5,279,376.35","155,520,152.92","245,970,223.67","17,503,871.50","728,262.21","264,202,357.38",0.91,0.08,0.01,0.84,0.12,0.03,0.93,0.07,0.00


## Export the data

In [34]:
%%notify
writer = pd.ExcelWriter("data/national_analysis.xlsx")
contributions_by_party_full_cycles.to_excel(writer, "contributions_party_full_cycles", index=False)
contributions_by_party.to_excel(writer, "contributions_by_party", index=False)
contributions_by_office.to_excel(writer, "contributions_by_office", index=False)
contributions_by_redistricting.to_excel(writer, "contributions_by_redistricting", index=False)
contributions_by_status.to_excel(writer, "contributions_by_status", index=False)
writer.save()

<IPython.core.display.Javascript object>